Hey guys!

After the competition closed, I want to share my idea of filling missing with you.
(The original train data have too mcuh missing...)

The main idea is "time-series similarity between meters in single site".
(for example, meter_A's pattern is very similar to meter_B in site1, and I do believe that some of you have noticed that)
By well utilizing such similarity, we can find out those missing data!

In [ ]:
!pip install kneed

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from pathlib import Path

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose

from kneed import KneeLocator

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def MinMaxScaler(data):
    return (data-np.min(data))/(np.max(data)-np.min(data))

def Kmeans_clustering(df, clusterNum, max_iter, n_jobs):
    scaler = StandardScaler()
    scaler.fit(df)
    df_std = pd.DataFrame(data=scaler.transform(df), columns=df.columns, index=df.index)
    km_model = KMeans(n_clusters=clusterNum, max_iter=max_iter, n_jobs=n_jobs, random_state=666)
    km_model = km_model.fit(df_std)
    clusterdf= pd.DataFrame(data=km_model.labels_, columns=['ClusterNo'])
    clusterdf.index = df.index
    return clusterdf

def Kmeans_bestClusterNum(df, range_min, range_max, max_iter, n_jobs):
    silhouette_avgs = []
    sum_of_squared_distances = []
    
    ks = range(range_min,range_max+1)
    for k in ks:
        kmeans_fit = KMeans(n_clusters = k, n_jobs=n_jobs, max_iter=max_iter, random_state=666).fit(df)
        cluster_labels = kmeans_fit.labels_
        sum_of_squared_distances.append(kmeans_fit.inertia_)
        
    kn = KneeLocator(list(ks), sum_of_squared_distances, S=1.0, curve='convex', direction='decreasing')  
    plt.xlabel('k')
    plt.ylabel('sum_of_squared_distances')
    plt.title('The Elbow Method showing the optimal k')
    plt.plot(ks, sum_of_squared_distances, 'bx-')
    plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
    print('Optimal clustering number:'+str(kn.knee))
    print('----------------------------')    
    
    return kn.knee

In [ ]:
%%time
root = Path('../input/ashrae-feather-format-for-fast-loading')

train_df = pd.read_feather(root/'train.feather')
test_df = pd.read_feather(root/'test.feather')
weather_train_df = pd.read_feather(root/'weather_train.feather')
weather_test_df = pd.read_feather(root/'weather_test.feather')
building_meta_df = pd.read_feather(root/'building_metadata.feather')

In [ ]:
#Let's just take site1 as example
train_df = train_df.merge(building_meta_df, on='building_id')
train_df = train_df[train_df['site_id']==1]

train_df['merged_id'] = 'site'+train_df['site_id'].astype(str)+'_'\
                        +'bldg'+train_df['building_id'].astype(str)+'_'\
                        +'meter'+train_df['meter'].astype(str)
train_df = train_df[['timestamp', 'site_id', 'building_id', 'meter' ,'meter_reading', 'merged_id']]
train_df = train_df.set_index('timestamp')

train_df = train_df.sort_values(['merged_id','timestamp'])
train_df

In [ ]:
train_df_pivot = train_df.pivot_table(values='meter_reading', index='timestamp', columns='merged_id')
train_df_pivot

In [ ]:
df_PM_temp = train_df_pivot.copy()
df_PM_temp = (df_PM_temp-df_PM_temp.mean())/df_PM_temp.std()
STL_decomp = seasonal_decompose(df_PM_temp.fillna(0), model='additive',freq=24*7,extrapolate_trend=1)

df_seasonal_temp=STL_decomp.seasonal.iloc[24*10:24*17,:]
df_seasonal_temp = df_seasonal_temp.T

df_seasonal_temp['ClusterNo'] = Kmeans_clustering(df=df_seasonal_temp, clusterNum=5, max_iter=100000, n_jobs=-1)

for ClusterNo in df_seasonal_temp['ClusterNo'].unique():
    df_plot = df_seasonal_temp[df_seasonal_temp['ClusterNo']==ClusterNo].T.drop('ClusterNo')
    print('ClusterNo: ' + str(ClusterNo))    
    print('Amount of meters: ' + str(len(df_plot.T)))
    df_plot.plot(figsize=(15,5),color='black',alpha=0.1,legend=False)
    plt.show()
    print('---------------------------------------------------------------------------------------------------')    

In [ ]:
df_PM_temp = train_df_pivot.copy()
df_PM_temp = (df_PM_temp-df_PM_temp.mean())/df_PM_temp.std()
STL_decomp = seasonal_decompose(df_PM_temp.fillna(0), model='additive',freq=24*7,extrapolate_trend=1)

df_trend_temp = STL_decomp.trend
#df_trend_temp = df_trend_temp.loc['2016']
df_trend_temp = df_trend_temp.T

df_trend_temp['ClusterNo'] = Kmeans_clustering(df=df_trend_temp, clusterNum=5, max_iter=100000, n_jobs=-1)

for ClusterNo in df_trend_temp['ClusterNo'].unique():
    df_plot = df_trend_temp[df_trend_temp['ClusterNo']==ClusterNo].T.drop('ClusterNo')
    print('ClusterNo: ' + str(ClusterNo))    
    print('Amount of meters: ' + str(len(df_plot.T)))
    df_plot.plot(figsize=(15,5),color='black',alpha=0.1,legend=False,ylim=(-3,3))
    plt.show()
    print('---------------------------------------------------------------------------------------------------')

Take a look of "site1_bldg105_meter0", we can observe abnormal constant in some days.

In [ ]:
#Graph of original data: "site1_bldg105_meter0"
train_df_pivot['site1_bldg105_meter0'].iplot(kind='scatter')

In [ ]:
#Let's clean these abnormal days by checking std_daily==0

list_powerMeter = list(train_df_pivot.columns)

for name_meter in list_powerMeter:
    df_daily = train_df_pivot[name_meter].copy()
    df_daily = df_daily.resample('D').std()

    list_abnormalDate = list(df_daily[df_daily==0].index.strftime('%Y-%m-%d'))
    train_df_pivot['Date'] = pd.to_datetime(train_df_pivot.index.date)

    train_df_pivot.loc[train_df_pivot['Date'].isin(list_abnormalDate), name_meter] = np.nan
    train_df_pivot = train_df_pivot.drop('Date', axis=1)  

Successfully clean these abnormal days!

In [ ]:
#Graph of cleand data: "site1_bldg105_meter0"
train_df_pivot['site1_bldg105_meter0'].iplot(kind='scatter')

Now, let's take "site1_bldg105_meter0" as example for fillling missing

In [ ]:
#Prepare data: "site1_bldg105_meter0"
example_df = train_df_pivot.copy()
example_df = example_df.dropna(axis=1) #Only take non-missing meter readings as inputs

example_df['elec_meas'] = train_df_pivot['site1_bldg105_meter0'].copy() #Take "site1_bldg105_meter0" as target

traindata = example_df[~example_df['elec_meas'].isna()]
testdata = example_df[example_df['elec_meas'].isna()]

train_labels = traindata['elec_meas']

train_features = traindata.drop('elec_meas', axis=1)
test_features = testdata.drop('elec_meas', axis=1)

In [ ]:
LGB_model = lgb.LGBMRegressor()
LGB_model.fit(train_features, train_labels)

testdata['elec_pred'] = LGB_model.predict(test_features)

# Use the forest's predict method on the train data
example_df['elec_pred'] = LGB_model.predict(example_df.drop('elec_meas', axis=1))

Boom! Here's the meter reading after filling missing😎

In [ ]:
example_df[['elec_meas', 'elec_pred']].iplot(kind='scatter')

Hope you guys enjoy!